In [ ]:
# --- Cell 0: Load Source Data ---
%matplotlib widget

import opym
from pathlib import Path
import importlib
import sys
import numpy as np
import matplotlib.pyplot as plt
import traceback

import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display, clear_output

# Reload modules
try:
    importlib.reload(opym.dataloader)
    importlib.reload(opym.petakit)
    importlib.reload(opym.utils)
    importlib.reload(opym.viewer)  # <-- ADDED for viewer cells
    importlib.reload(opym)
    print("Reloaded opym library.")
except Exception as e:
    print(f"Note: Could not reload all modules: {e}")


# --- Create Interactive Widgets ---
print("\n1. Select your source data directory (OPM or LLSM).")
print("2. Choose the channel to use for the preview MIP.")
print("3. Click 'Load Data'.")

dir_chooser = FileChooser(
    path="/mmfs2/scratch/SDSMT.LOCAL/bscott/DataUpload",
    title="<b>Select Source TIFF Directory (OPM or LLSM):</b>",
    show_only_dirs=True
)

# --- NEW: Widget to select preview channel ---
preview_channel_widget = widgets.IntText(
    value=2,
    description="Preview Channel:",
    style={"description_width": "initial"},
)
# --- END NEW ---

load_button = widgets.Button(
    description="Load Data", button_style="primary"
)
load_output = widgets.Output()

# --- Global variables to hold data and parameters ---
source_processed_dir: Path | None = None
mip_original: np.ndarray | None = None
stack_original: np.ndarray | None = None  # <-- ADDED for orthoviews
vmin_original: float = 0
vmax_original: float = 1
data_type_source: opym.MicroscopyDataType = "UNKNOWN"  # Use type from utils
# --- Globals for OPM data (used in Cell 1) ---
get_stack_source = None
t_min_source = 0
t_max_source = 0
c_min_source = 0
c_max_source = 0
base_name_source = ""

# Default physical params (from config.json)
default_xy_size = 0.136
default_z_step = 1.0
default_angle = 21.72


def on_load_button_clicked(b):
    global source_processed_dir, mip_original, vmin_original, vmax_original
    global get_stack_source, t_min_source, t_max_source, c_min_source
    global c_max_source, base_name_source, data_type_source
    global stack_original  # <-- ADDED for orthoviews

    with load_output:
        clear_output(wait=True)
        if not dir_chooser.value:
            print("❌ ERROR: No directory selected.")
            return

        source_processed_dir = Path(dir_chooser.value)
        preview_channel = preview_channel_widget.value

        try:
            # --- MODIFIED: Use opym.detect_microscopy_data_type ---
            data_type_source = opym.detect_microscopy_data_type(
                source_processed_dir
            )

            if data_type_source == "OPM":
                print(
                    f"--- Loading OPM data from: {source_processed_dir.name} ---"
                )

                # ---
                # FIX 1: Unpack all 9 values for OPM
                # ---
                (
                    get_stack,
                    t_min,
                    t_max,
                    c_min,
                    c_max,
                    z_max,
                    y,
                    x,
                    base_name,
                ) = opym.load_tiff_series(source_processed_dir)

                # Store globals for Cell 1
                get_stack_source = get_stack
                t_min_source = t_min
                t_max_source = t_max
                c_min_source = c_min
                c_max_source = c_max
                base_name_source = base_name

                # --- FIX: Load the *user-selected* channel for preview ---
                if not c_min <= preview_channel <= c_max:
                    print(
                        f"⚠️ Warning: Channel {preview_channel} not in range "
                        f"({c_min}-{c_max}). Defaulting to C={c_min}."
                    )
                    preview_channel = c_min

                print(f"  Generating MIP from T={t_min}, C={preview_channel}...")
                stack_original = get_stack(t_min, preview_channel)
                mip_original = np.max(stack_original, axis=0)
                # --- END FIX ---

            elif data_type_source == "LLSM":
                print(
                    "--- Loading LLSM preview from: "
                    f"{source_processed_dir.name} ---"
                )

                # ---
                # FIX 2: Use the correct function opym.load_llsm_tiff_series
                # ---
                (
                    get_stack,
                    t_min,
                    t_max,
                    c_min,
                    c_max,
                    z_max,
                    y,
                    x,
                    base_name,
                ) = opym.load_llsm_tiff_series(source_processed_dir)
                
                # ---
                # FIX 2.1: Add logic to generate the MIP from the loaded data
                # ---
                if not c_min <= preview_channel <= c_max:
                    print(
                        f"⚠️ Warning: Channel {preview_channel} not in range "
                        f"({c_min}-{c_max}). Defaulting to C={c_min}."
                    )
                    preview_channel = c_min
                
                print(f"  Generating MIP from T={t_min}, C={preview_channel}...")
                stack_original = get_stack(t_min, preview_channel)
                mip_original = np.max(stack_original, axis=0)
                # --- END FIX 2.1 ---

                # Store globals
                get_stack_source = None  # No loader function needed for LLSM
                base_name_source = ""  # Will be determined in Cell 1
                print(f"  Previewing {base_name}")

            else:
                print("❌ ERROR: Unknown data type in directory.")
                print(
                    "  Could not find OPM (*_T..._C...) or "
                    "LLSM (*_Cam..._stack...) files."
                )
                data_type_source = "UNKNOWN"
                return

            # --- COMMON: Calculate and store the original MIP ---
            if mip_original is None:
                # This should be logically impossible, but satisfies the linter
                print("❌ ERROR: MIP calculation failed unexpectedly.", file=sys.stderr)
                data_type_source = "UNKNOWN" # Reset state
                return
            vmin_original, vmax_original = np.percentile(
                mip_original, [1, 99.9]
            )
            if vmin_original >= vmax_original:
                vmax_original = np.max(mip_original)

            print(f"\n✅ {data_type_source} data loaded. MIP calculated.")
            print("You can now run Cell 1 to tune parameters.")

        except FileNotFoundError as e:
            print(f"\n❌ ERROR: Could not load data.")
            print(f"  Details: {e}")
            data_type_source = "UNKNOWN"
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)
            traceback.print_exc(file=sys.stderr)
            data_type_source = "UNKNOWN"

load_button.on_click(on_load_button_clicked)
# --- MODIFIED: Added preview_channel_widget to display ---
display(
    widgets.VBox(
        [dir_chooser, preview_channel_widget, load_button, load_output]
    )
)

In [ ]:
# --- Cell 1: Interactive Parameter Tuner ---

import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt
import traceback  # Import for error logging

# --- Global to hold parameters for viewer cells ---
processed_viewer_params = None

# --- Create Parameter Sliders ---
angle_slider = widgets.FloatSlider(
    value=default_angle,
    min=10.0,  # <-- Widened range
    max=40.0,  # <-- Widened range
    step=0.01,
    description='Sheet Angle:',
    readout_format='.2f',
    style={'description_width': 'initial'},
)
z_step_slider = widgets.FloatSlider(
    value=default_z_step,
    min=0.05,  # <-- Widened range
    max=5.0,   # <-- Widened range
    step=0.05,
    description='Z-Step (um):',
    readout_format='.2f',
    style={'description_width': 'initial'},
)
xy_size_slider = widgets.FloatSlider(
    value=default_xy_size,
    min=0.05,
    max=0.2,
    step=0.001,
    description='XY Pixel Size (um):',
    readout_format='.3f',
    style={'description_width': 'initial'},
)

# --- Create Buttons and Output Areas ---
run_view_button = widgets.Button(
    description="Run & View Processed Stack", button_style="success"
)
log_output = widgets.Output(layout={'border': '1px solid black', 'height': '200px', 'overflow_y': 'scroll'})
view_output = widgets.Output() # This will hold the plot

# --- Define Button Click Handler ---
def on_run_view_button_clicked(b):
    global processed_viewer_params # <-- ADDED for viewer cells
    run_view_button.disabled = True
    
    with log_output:
        clear_output(wait=True)
    with view_output:
        clear_output(wait=True) # Clear previous plot

    try:
        # Check if globals from Cell 0 are loaded
        if (source_processed_dir is None 
            or mip_original is None 
            or stack_original is None): # <-- ADDED check
            with log_output:
                print("❌ ERROR: Source data not loaded. Please run Cell 0 first.")
            return

        # --- MODIFIED: Check global data type var ---
        if data_type_source == "UNKNOWN":
            with log_output:
                print(
                    "❌ ERROR: Data type is UNKNOWN. "
                    "Please re-run Cell 0 and select a valid directory."
                )
            return
        # --- END MODIFIED ---

        # 1. Get all parameters from widgets
        angle = angle_slider.value
        z_step = z_step_slider.value
        xy_size = xy_size_slider.value
        
        with log_output:
            print(f"Processing {data_type_source} data with:")
            print(f"  Angle={angle}, Z-Step={z_step}, XY-Size={xy_size}...")

        # 2. Create unique directory name for this run
        dsr_dir_name = f"DSR_a{angle:.2f}_z{z_step:.2f}_xy{xy_size:.3f}"
        ds_dir_name = f"DS_a{angle:.2f}_z{z_step:.2f}_xy{xy_size:.3f}"
        
        # 3. --- MODIFIED: Generalized Processing ---
        # Collect all parameters into a dict
        processing_params = {
            "ds_dir_name": ds_dir_name,
            "dsr_dir_name": dsr_dir_name,
            "sheet_angle_deg": angle,
            "z_step_um": z_step,
            "xy_pixel_size": xy_size,
            "save_mip": True,
            # Add any other ppk5d flags you want to control here
        }

        if data_type_source == "OPM":
            print("--- Running OPM processing (opym.run_petakit_processing) ---")
            opym.run_petakit_processing(
                source_processed_dir, **processing_params
            )
        elif data_type_source == "LLSM":
            print("--- Running LLSM processing (opym.run_llsm_petakit_processing) ---")
            opym.run_llsm_petakit_processing(
                source_processed_dir, **processing_params
            )
        # --- END MODIFIED ---
        
        # 4. Load the processed stack
        # This logic is THE SAME, as ppk5d *always*
        # writes to the _T..._C... format, which opym.load_tiff_series can read.
        output_path = source_processed_dir / dsr_dir_name
        with log_output:
            print(f"✅ Processing complete. Loading results from: {output_path.name}")

        # opym.load_tiff_series works on the *output* of ppk5d
        (
            get_stack_proc,
            t_min_proc,
            t_max_proc,
            c_min_proc,
            c_max_proc,
            z_max_proc,
            y_proc,
            x_proc,
            bn_proc,
        ) = opym.load_tiff_series(output_path)
        
        # --- ADDED: Store parameters for viewer cells ---
        processed_viewer_params = (
            get_stack_proc,
            t_max_proc,
            z_max_proc,
            c_max_proc,
            y_proc,
            x_proc,
        )
        # --- END ADDED ---

        # ---
        # FIX: Load the stack corresponding to the *preview channel*
        # from Cell 0, not just c_min_proc.
        # ---
        preview_channel_to_load = preview_channel_widget.value

        # Check if this channel is valid in the *processed* data
        if not c_min_proc <= preview_channel_to_load <= c_max_proc:
            with log_output:
                print(
                    f"⚠️ Warning: Preview channel {preview_channel_to_load} "
                    f"not in processed range ({c_min_proc}-{c_max_proc})."
                )
                print(f"   Defaulting to processed C={c_min_proc}.")
            preview_channel_to_load = c_min_proc

        with log_output:
            print(
                f"   Loading processed T={t_min_proc}, "
                f"C={preview_channel_to_load} for MIP."
            )

        stack_processed = get_stack_proc(t_min_proc, preview_channel_to_load)
        # --- END FIX ---
        
        # 5. --- MODIFIED: Calculate all Orthoview MIPS ---
        
        # Original (from globals)
        mip_original_xy = mip_original # (Y, X)
        mip_original_xz = np.max(stack_original, axis=1) # (Z, X)
        mip_original_yz = np.max(stack_original, axis=2) # (Z, Y)
        # vmin_original, vmax_original are already global

        # Processed (from local vars)
        mip_processed_xy = np.max(stack_processed, axis=0) # (Y, X)
        mip_processed_xz = np.max(stack_processed, axis=1) # (Z, X)
        mip_processed_yz = np.max(stack_processed, axis=2) # (Z, Y)
        
        vmin_proc, vmax_proc = np.percentile(mip_processed_xy, [1, 99.9])
        if vmin_proc >= vmax_proc:
            vmax_proc = np.max(mip_processed_xy)
        # --- END MODIFIED ---

        # 6. --- MODIFIED: Display the 2x3 Orthoview results ---
        with view_output:
            fig, axes = plt.subplots(2, 3, figsize=(15, 8)) # (rows, cols)
            
            # --- Row 0: Original Data ---
            axes[0, 0].imshow(mip_original_xy, cmap='gray', vmin=vmin_original, vmax=vmax_original)
            axes[0, 0].set_title(f"Original XY (Shape: {mip_original_xy.shape})")
            axes[0, 0].axis('off')
            
            axes[0, 1].imshow(mip_original_xz, cmap='gray', vmin=vmin_original, vmax=vmax_original)
            axes[0, 1].set_title(f"Original XZ (Shape: {mip_original_xz.shape})")
            axes[0, 1].axis('off')
            axes[0, 1].set_aspect('auto') # XZ aspect is often different
            
            axes[0, 2].imshow(mip_original_yz, cmap='gray', vmin=vmin_original, vmax=vmax_original)
            axes[0, 2].set_title(f"Original YZ (Shape: {mip_original_yz.shape})")
            axes[0, 2].axis('off')
            axes[0, 2].set_aspect('auto') # YZ aspect is often different

            # --- Row 1: Processed Data ---
            axes[1, 0].imshow(mip_processed_xy, cmap='gray', vmin=vmin_proc, vmax=vmax_proc)
            axes[1, 0].set_title(f"Processed XY (Shape: {mip_processed_xy.shape})")
            axes[1, 0].axis('off')
            
            axes[1, 1].imshow(mip_processed_xz, cmap='gray', vmin=vmin_proc, vmax=vmax_proc)
            axes[1, 1].set_title(f"Processed XZ (Shape: {mip_processed_xz.shape})")
            axes[1, 1].axis('off')
            axes[1, 1].set_aspect('auto') # Processed aspect is also different
            
            axes[1, 2].imshow(mip_processed_yz, cmap='gray', vmin=vmin_proc, vmax=vmax_proc)
            axes[1, 2].set_title(f"Processed YZ (Shape: {mip_processed_yz.shape})")
            axes[1, 2].axis('off')
            axes[1, 2].set_aspect('auto') # Processed aspect is also different
            
            plt.tight_layout()
            plt.show()
            
            with log_output:
                print("\n✅ Orthoviews displayed.")
                print("You can now run Cell 2 or 3 to view the processed stack interactively.")
        # --- END MODIFIED ---

    except Exception as e:
        with log_output:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)
            traceback.print_exc(file=sys.stderr)
    finally:
        # Re-enable the button
        run_view_button.disabled = False


# --- Link button and display UI ---
run_view_button.on_click(on_run_view_button_clicked)

param_box = widgets.VBox([
    angle_slider,
    z_step_slider,
    xy_size_slider
])

display(widgets.VBox([param_box, run_view_button, log_output, view_output]))

In [ ]:
# --- Cell 3: Launch the composite viewer ---
# (Run this cell after running Cell 1)

try:
    if processed_viewer_params is None:
        raise NameError
    opym.composite_viewer(*processed_viewer_params)
except NameError:
    print("❌ ERROR: 'processed_viewer_params' not found. Please run Cell 1 to process data first.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")